# Prototype D5 + Common Core join

In [34]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [2]:
d5 = pd.read_csv('../data/raw/D5 SHSAT Registrations and Testers.csv')
d5.head()

,DBN,School name,Year of SHST,Grade level,Enrollment on 10/31,Number of students who registered for the SHSAT,Number of students who took the SHSAT
0,05M046,P.S. 046 Arthur Tappan,2013,8,91,31,14
1,05M046,P.S. 046 Arthur Tappan,2014,8,95,26,7
2,05M046,P.S. 046 Arthur Tappan,2015,8,73,21,10
3,05M046,P.S. 046 Arthur Tappan,2016,8,56,20,8
4,05M123,P.S. 123 Mahalia Jackson,2013,8,62,31,14


In [3]:
tests = pd.read_pickle('../data/pre/test_results.pkl')
tests.head()

Charter School?  Number Tested - ELA  \
DBN    Grade Year                                         
01M015 3     2013                0                 27.0   
             2014                0                 18.0   
             2015                0                 16.0   
             2016                0                 20.0   
             2017                0                 27.0   

                   Mean Scale Score - ELA  # Level 1 - ELA  % Level 1 - ELA  \
DBN    Grade Year                                                             
01M015 3     2013               -0.627863             14.0         51.85185   
             2014               -0.867271             10.0         55.55556   
             2015               -1.055963              9.0         56.25000   
             2016               -0.444600             10.0         50.00000   
             2017                0.120021             10.0         37.03704   

                   # Level 2 - ELA  % Level 2 - ELA  # Level 3 - ELA  \
DBN    Grade Year                                                      
01M015 3     2013             11.0         0.407407              2.0   
             2014              8.0         0.444444              0.0   
             2015              5.0         0.312500              2.0   
             2016              6.0         0.300000              4.0   
             2017              8.0         0.296296              7.0   

                   % Level 3 - ELA  # Level 4 - ELA         ...          \
DBN    Grade Year                                           ...           
01M015 3     2013         0.074074              0.0         ...           
             2014         0.000000              0.0         ...           
             2015         0.125000              0.0         ...           
             2016         0.200000              0.0         ...           
             2017         0.259259              2.0         ...           

                   # Level 1 - Math  % Level 1 - Math  # Level 2 - Math  \
DBN    Grade Year                                                         
01M015 3     2013              16.0          59.25926              11.0   
             2014               6.0          33.33333               9.0   
             2015              10.0          58.82353               4.0   
             2016              13.0          61.90476               4.0   
             2017               8.0          27.58621               9.0   

                   % Level 2 - Math  # Level 3 - Math  % Level 3 - Math  \
DBN    Grade Year                                                         
01M015 3     2013          0.407407               0.0          0.000000   
             2014          0.500000               2.0          0.111111   
             2015          0.235294               2.0          0.117647   
             2016          0.190476               4.0          0.190476   
             2017          0.310345               7.0          0.241379   

                   # Level 4 - Math  % Level 4 - Math  # Level 3+4 - Math  \
DBN    Grade Year                                                           
01M015 3     2013               0.0          0.000000                 0.0   
             2014               1.0          0.055556                 3.0   
             2015               1.0          0.058824                 3.0   
             2016               0.0          0.000000                 4.0   
             2017               5.0          0.172414                12.0   

                   % Level 3+4 - Math  
DBN    Grade Year                      
01M015 3     2013             0.00000  
             2014            16.66667  
             2015            17.64706  
             2016            19.04762  
             2017            41.37931  

[5 rows x 25 columns]

# Preprocess D5

In [30]:
df = pd.read_csv('../data/raw/D5 SHSAT Registrations and Testers.csv')

# keep grade 8
df = df[df['Grade level'] == 8]
df = df.drop('Grade level', axis=1)

# rename columns
df = df.rename({
    'School name': 'School Name',
    'Year of SHST': 'Year',
    'Number of students who registered for the SHSAT': '# SHSAT Registrants',
    'Number of students who took the SHSAT': '# SHSAT Testers'
}, axis=1)

# create columns
df['% SHSAT Registrants'] = df['# SHSAT Registrants'] / df['Enrollment on 10/31']
df['% SHSAT Testers'] = df['# SHSAT Testers'] / df['Enrollment on 10/31']

# clip percentage values
df['% SHSAT Registrants'] = np.clip(df['% SHSAT Registrants'], 0.0, 1.0)
df['% SHSAT Testers'] = np.clip(df['% SHSAT Testers'], 0.0, 1.0)

# set index
df = df.set_index(['DBN', 'Year'])

# export
centralharlem = df
del df

centralharlem.head()

School Name  Enrollment on 10/31  \
DBN    Year                                                  
05M046 2013    P.S. 046 Arthur Tappan                   91   
       2014    P.S. 046 Arthur Tappan                   95   
       2015    P.S. 046 Arthur Tappan                   73   
       2016    P.S. 046 Arthur Tappan                   56   
05M123 2013  P.S. 123 Mahalia Jackson                   62   

             # SHSAT Registrants  # SHSAT Testers  % SHSAT Registrants  \
DBN    Year                                                              
05M046 2013                   31               14             0.340659   
       2014                   26                7             0.273684   
       2015                   21               10             0.287671   
       2016                   20                8             0.357143   
05M123 2013                   31               14             0.500000   

             % SHSAT Testers  
DBN    Year                   
05M046 2013         0.153846  
       2014         0.073684  
       2015         0.136986  
       2016         0.142857  
05M123 2013         0.225806

# Preprocess tests

In [31]:
tests = pd.read_pickle('../data/pre/test_results.pkl')

# keep 7th grade
tests = tests.reset_index()
tests = tests[tests['Grade'] == 7]
tests = tests.drop('Grade', axis=1)
tests = tests.set_index(['DBN', 'Year'])
tests.head()

Charter School?  Number Tested - ELA  Mean Scale Score - ELA  \
DBN    Year                                                                 
01M015 2013                0                  0.0                     NaN   
       2014                0                  0.0                     NaN   
       2015                0                  0.0                     NaN   
       2016                0                  0.0                     NaN   
       2017                0                  0.0                     NaN   

             # Level 1 - ELA  % Level 1 - ELA  # Level 2 - ELA  \
DBN    Year                                                      
01M015 2013              NaN              NaN              NaN   
       2014              NaN              NaN              NaN   
       2015              NaN              NaN              NaN   
       2016              NaN              NaN              NaN   
       2017              NaN              NaN              NaN   

             % Level 2 - ELA  # Level 3 - ELA  % Level 3 - ELA  \
DBN    Year                                                      
01M015 2013              NaN              NaN              NaN   
       2014              NaN              NaN              NaN   
       2015              NaN              NaN              NaN   
       2016              NaN              NaN              NaN   
       2017              NaN              NaN              NaN   

             # Level 4 - ELA         ...          # Level 1 - Math  \
DBN    Year                          ...                             
01M015 2013              NaN         ...                       NaN   
       2014              NaN         ...                       NaN   
       2015              NaN         ...                       NaN   
       2016              NaN         ...                       NaN   
       2017              NaN         ...                       NaN   

             % Level 1 - Math  # Level 2 - Math  % Level 2 - Math  \
DBN    Year                                                         
01M015 2013               NaN               NaN               NaN   
       2014               NaN               NaN               NaN   
       2015               NaN               NaN               NaN   
       2016               NaN               NaN               NaN   
       2017               NaN               NaN               NaN   

             # Level 3 - Math  % Level 3 - Math  # Level 4 - Math  \
DBN    Year                                                         
01M015 2013               NaN               NaN               NaN   
       2014               NaN               NaN               NaN   
       2015               NaN               NaN               NaN   
       2016               NaN               NaN               NaN   
       2017               NaN               NaN               NaN   

             % Level 4 - Math  # Level 3+4 - Math  % Level 3+4 - Math  
DBN    Year                                                            
01M015 2013               NaN                 NaN                 NaN  
       2014               NaN                 NaN                 NaN  
       2015               NaN                 NaN                 NaN  
       2016               NaN                 NaN                 NaN  
       2017               NaN                 NaN                 NaN  

[5 rows x 25 columns]

# Join DataFrames

In [36]:
joined = tests.join(centralharlem, how='right')
joined.head()

Charter School?  Number Tested - ELA  Mean Scale Score - ELA  \
DBN    Year                                                                 
05M046 2013              0.0                 84.0               -1.117422   
       2014              0.0                 85.0               -1.628713   
       2015              0.0                 66.0               -1.380656   
       2016              0.0                 58.0               -1.103425   
05M123 2013              0.0                 55.0               -1.045237   

             # Level 1 - ELA  % Level 1 - ELA  # Level 2 - ELA  \
DBN    Year                                                      
05M046 2013             47.0         55.95238             29.0   
       2014             57.0         67.05882             22.0   
       2015             41.0         62.12121             16.0   
       2016             39.0         67.24138             14.0   
05M123 2013             30.0         54.54545             20.0   

             % Level 2 - ELA  # Level 3 - ELA  % Level 3 - ELA  \
DBN    Year                                                      
05M046 2013         0.345238              7.0         0.083333   
       2014         0.258823              6.0         0.070588   
       2015         0.242424              9.0         0.136364   
       2016         0.241379              2.0         0.034483   
05M123 2013         0.363636              5.0         0.090909   

             # Level 4 - ELA  % Level 4 - ELA  # Level 3+4 - ELA  \
DBN    Year                                                        
05M046 2013              1.0         0.011905                8.0   
       2014              0.0         0.000000                6.0   
       2015              0.0         0.000000                9.0   
       2016              3.0         0.051724                5.0   
05M123 2013              0.0         0.000000                5.0   

             % Level 3+4 - ELA  Number Tested - Math  Mean Scale Score - Math  \
DBN    Year                                                                     
05M046 2013           9.523810                  88.0                -0.744606   
       2014           7.058824                  87.0                -1.012577   
       2015          13.636360                  67.0                -0.977286   
       2016           8.620690                  58.0                -1.679011   
05M123 2013           9.090909                  61.0                -1.690491   

             # Level 1 - Math  % Level 1 - Math  # Level 2 - Math  \
DBN    Year                                                         
05M046 2013              50.0          56.81818              31.0   
       2014              56.0          64.36782              26.0   
       2015              39.0          58.20896              23.0   
       2016              48.0          82.75862              10.0   
05M123 2013              47.0          77.04918              11.0   

             % Level 2 - Math  # Level 3 - Math  % Level 3 - Math  \
DBN    Year                                                         
05M046 2013          0.352273               7.0          0.079545   
       2014          0.298851               5.0          0.057471   
       2015          0.343284               4.0          0.059701   
       2016          0.172414               0.0          0.000000   
05M123 2013          0.180328               3.0          0.049180   

             # Level 4 - Math  % Level 4 - Math  # Level 3+4 - Math  \
DBN    Year                                                           
05M046 2013               0.0          0.000000                 7.0   
       2014               0.0          0.000000                 5.0   
       2015               1.0          0.014925                 5.0   
       2016               0.0          0.000000                 0.0   
05M123 2013               0.0          0.000000                 3.0   

             % Level 3+4 - Math     

In [37]:
# null analysis
joined[joined.isnull().any(axis=1)]

Charter School?  Number Tested - ELA  Mean Scale Score - ELA  \
DBN    Year                                                                 
05M367 2013              NaN                  NaN                     NaN   
       2014              NaN                  NaN                     NaN   
05M469 2013              NaN                  NaN                     NaN   
       2014              NaN                  NaN                     NaN   

             # Level 1 - ELA  % Level 1 - ELA  # Level 2 - ELA  \
DBN    Year                                                      
05M367 2013              NaN              NaN              NaN   
       2014              NaN              NaN              NaN   
05M469 2013              NaN              NaN              NaN   
       2014              NaN              NaN              NaN   

             % Level 2 - ELA  # Level 3 - ELA  % Level 3 - ELA  \
DBN    Year                                                      
05M367 2013              NaN              NaN              NaN   
       2014              NaN              NaN              NaN   
05M469 2013              NaN              NaN              NaN   
       2014              NaN              NaN              NaN   

             # Level 4 - ELA  % Level 4 - ELA  # Level 3+4 - ELA  \
DBN    Year                                                        
05M367 2013              NaN              NaN                NaN   
       2014              NaN              NaN                NaN   
05M469 2013              NaN              NaN                NaN   
       2014              NaN              NaN                NaN   

             % Level 3+4 - ELA  Number Tested - Math  Mean Scale Score - Math  \
DBN    Year                                                                     
05M367 2013                NaN                   NaN                      NaN   
       2014                NaN                   NaN                      NaN   
05M469 2013                NaN                   NaN                      NaN   
       2014                NaN                   NaN                      NaN   

             # Level 1 - Math  % Level 1 - Math  # Level 2 - Math  \
DBN    Year                                                         
05M367 2013               NaN               NaN               NaN   
       2014               NaN               NaN               NaN   
05M469 2013               NaN               NaN               NaN   
       2014               NaN               NaN               NaN   

             % Level 2 - Math  # Level 3 - Math  % Level 3 - Math  \
DBN    Year                                                         
05M367 2013               NaN               NaN               NaN   
       2014               NaN               NaN               NaN   
05M469 2013               NaN               NaN               NaN   
       2014               NaN               NaN               NaN   

             # Level 4 - Math  % Level 4 - Math  # Level 3+4 - Math  \
DBN    Year                                                           
05M367 2013               NaN               NaN                 NaN   
       2014               NaN               NaN                 NaN   
05M469 2013               NaN               NaN                 NaN   
       2014               NaN               NaN                 NaN   

             % Level 3+4 - Math  \
DBN    Year                       
05M367 2013                 NaN   
       2014                 NaN   
05M469 2013                 NaN   
       2014                 NaN   

                                                   School Name  \
DBN    Year                                                      
05M367 2013  Academy for Social Action: A College Board School   
       2014  Academy for Social Action: A College Board School   
05M469 2013                            Choir Academy of Harlem   
       2014                            Choir Academy of Harlem

I will keep these rows as they mean something to the data.